In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lenskit as lk 
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv', encoding='latin-1')

In [3]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
2495,109487,Interstellar (2014),Sci-Fi|IMAX
2496,111362,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi
2497,111759,Edge of Tomorrow (2014),Action|Sci-Fi|IMAX
2498,112556,Gone Girl (2014),Drama|Thriller


In [4]:
print(ratings.shape)
ratings.head()

(264505, 4)


,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


**Association Rule Based**

In [5]:
ratings = ratings.pivot(index = 'movieId',  columns ='userId', values = 'rating')
ratings.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
movieId,,,,,,,,,,,,,,,,,,,,,
1,2.5,NaN,5.0,4.5,5.0,NaN,4.0,3.0,4.5,4.5,...,3.0,NaN,NaN,NaN,3.5,NaN,4.0,3.0,NaN,3.0
2,NaN,NaN,NaN,4.0,4.0,NaN,3.0,3.0,NaN,NaN,...,NaN,3.5,NaN,3.5,NaN,NaN,2.0,1.0,NaN,NaN
3,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,3.5,NaN,3.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ratings = ratings.rename(index=movies.set_index('movieId')['title'])
ratings = ratings.transpose()
ratings

movieId,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Sudden Death (1995),GoldenEye (1995),"American President, The (1995)",...,"Hunger Games: Catching Fire, The (2013)","Hobbit: The Desolation of Smaug, The (2013)","Wolf of Wall Street, The (2013)",Her (2013),"Grand Budapest Hotel, The (2014)",Interstellar (2014),X-Men: Days of Future Past (2014),Edge of Tomorrow (2014),Gone Girl (2014),Guardians of the Galaxy (2014)
userId,,,,,,,,,,,,,,,,,,,,,
316,2.5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,4.5,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,2.5,3.0,4.5,4.0,NaN,NaN,3.0,4.5,3.5,3.0
910,5.0,4.0,3.5,NaN,3.5,3.5,NaN,NaN,NaN,4.0,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137805,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.5,3.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
138072,3.0,1.0,1.0,NaN,NaN,NaN,NaN,1.5,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#make dictionary of user and their movies
user_movies = {}
for i in range(len(ratings.index)):
    user_movies[ratings.index[i]] = []
    for j in range(len(ratings.columns)):
        if ratings.iloc[i,j] >= 3:
            user_movies[ratings.index[i]].append(ratings.columns[j])

In [ ]:
#delet user id and put lists in numpy array from user_movies
user_movies = np.array(list(user_movies.values()))
user_movies


In [11]:
len(user_movies)

862

In [13]:
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules

In [14]:
te = TransactionEncoder()
te_ary = te.fit(user_movies).transform(user_movies)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset

c:\Users\farsh\AppData\Local\Programs\PythonCodingPack\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),127 Hours (2010),...,Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,True,False,True,True,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
858,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,True,False,False,False
859,True,False,False,True,False,False,True,False,True,False,...,False,False,False,False,False,False,True,False,False,True
860,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
frequent_itemsets = apriori(dataset, min_support=0.3, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

c:\Users\farsh\AppData\Local\Programs\PythonCodingPack\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets,length
0,0.350348,(2001: A Space Odyssey (1968)),1
1,0.315545,(300 (2007)),1
2,0.305104,(Ace Ventura: Pet Detective (1994)),1
3,0.364269,(Aladdin (1992)),1
4,0.426914,(Alien (1979)),1
...,...,...,...
8379,0.343387,(Lord of the Rings: The Fellowship of the Ring...,7
8380,0.317865,(Lord of the Rings: The Fellowship of the Ring...,7
8381,0.303944,(Lord of the Rings: The Fellowship of the Ring...,7
8382,0.308585,(Lord of the Rings: The Fellowship of the Ring...,7


In [19]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

c:\Users\farsh\AppData\Local\Programs\PythonCodingPack\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
72998,"(Lord of the Rings: The Return of the King, Th...","(Terminator, The (1984), Lord of the Rings: Th...",0.363109,0.353828,0.301624,0.830671,2.347667,0.173146,3.816071,2,3
72979,"(Terminator, The (1984), Lord of the Rings: Th...","(Lord of the Rings: The Return of the King, Th...",0.353828,0.363109,0.301624,0.852459,2.347667,0.173146,4.316705,3,2
72981,"(Lord of the Rings: The Two Towers, The (2002)...","(Terminator, The (1984), Lord of the Rings: Th...",0.367749,0.350348,0.301624,0.820189,2.341070,0.172784,3.612977,3,2
72996,"(Terminator, The (1984), Lord of the Rings: Th...","(Lord of the Rings: The Two Towers, The (2002)...",0.350348,0.367749,0.301624,0.860927,2.341070,0.172784,4.546183,2,3
41509,"(Kill Bill: Vol. 2 (2004), Lord of the Rings: ...","(Lord of the Rings: The Two Towers, The (2002)...",0.352668,0.372390,0.307425,0.871711,2.340855,0.176095,4.892141,2,2
...,...,...,...,...,...,...,...,...,...,...,...
17376,"(Lord of the Rings: The Two Towers, The (2002))","(Pulp Fiction (1994), Matrix, The (1999))",0.638051,0.549884,0.421114,0.660000,1.200253,0.070260,1.323871,1,2
1880,"(Matrix, The (1999))",(Star Wars: Episode IV - A New Hope (1977)),0.732019,0.582367,0.511601,0.698891,1.200087,0.085298,1.386983,1,1
1881,(Star Wars: Episode IV - A New Hope (1977)),"(Matrix, The (1999))",0.582367,0.732019,0.511601,0.878486,1.200087,0.085298,2.205355,1,1
717,"(Dark Knight, The (2008))",(Fight Club (1999)),0.489559,0.649652,0.381671,0.779621,1.200059,0.063627,1.589751,1,1


In [22]:
print(rules.iloc[0,:])

antecedents           (Lord of the Rings: The Fellowship of the Ring...
consequents                                            (Aladdin (1992))
antecedent support                                             0.676334
consequent support                                             0.364269
support                                                        0.300464
confidence                                                     0.444254
lift                                                            1.21958
leverage                                                      0.0540964
conviction                                                      1.14392
antecedents_length                                                    1
consequents_length                                                    1
Name: 0, dtype: object


c:\Users\farsh\AppData\Local\Programs\PythonCodingPack\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Damped Mean**

In [21]:
#Movive mean rating
ratings2 = pd.read_csv('ratings.csv')
mean_rating = ratings2.groupby('movieId')['rating'].mean().sort_values(ascending=False)
mean_rating

movieId
318     4.364362
858     4.315848
1248    4.259259
2959    4.258503
7502    4.247423
          ...   
1453    1.642857
1707    1.596491
2383    1.573171
3593    1.456140
66      1.153846
Name: rating, Length: 2500, dtype: float64

In [22]:
#calculate Dampet Mean Rating
C = mean_rating.mean()  #mean of all mean ratings   
m = mean_rating.quantile(0.9) #90th percentile of mean ratings  
def weighted_rating(x, m=m, C=C):
    v = x['count']
    R = x['mean']
   
    return ((R*v + C*m) / (v+m)) #(v/(v+m) * R) + (m/(m+v) * C) #چرا اینا با هم برابرند؟

       

mean_rating = mean_rating.reset_index()
mean_rating['count'] = ratings2.groupby('movieId')['rating'].count()
mean_rating['mean'] = ratings2.groupby('movieId')['rating'].mean()
mean_rating['weighted_rating'] = mean_rating.apply(weighted_rating, axis=1)
mean_rating = mean_rating.sort_values('weighted_rating', ascending=False)
mean_rating.head()


,movieId,rating,count,mean,weighted_rating
318,2918,3.878613,564.0,4.364362,4.357477
858,46723,3.634375,448.0,4.315848,4.307616
1203,2355,3.480144,189.0,4.246032,4.228165
296,1288,3.891705,613.0,4.217781,4.212374
1221,85,3.472222,325.0,4.218462,4.208311
